In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow.keras as keras
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
import xgboost as xgb

In [3]:
import csv

In [4]:
def get_data(filename):
  with open(filename, "r") as training_file:
    csv_reader = csv.reader(training_file)
    next(csv_reader, None)  # skip the headers
    X = []
    Y = []
    for row in csv_reader:
        X.append(np.array(row[:-1]))
        Y.append(row[-1])

  X = np.array(X).astype(np.float32)
  Y = np.array(Y).astype(np.float32)
  return X, Y

In [5]:
X_features, Y = get_data('/content/drive/MyDrive/Colab Notebooks/vgg.csv')

In [6]:
X_features.shape

(6400, 15360)

In [7]:
Y[0]

0.0

In [8]:
kfold = KFold(n_splits=5, shuffle=True)

In [9]:
acc_per_fold = []
loss_per_fold = []
Y_preds = []
Y_actuals = []

In [10]:
from sklearn import metrics

In [12]:
%%time

fold_no = 1
num_classes = 2

for train, test in kfold.split(X_features,Y):

    model = xgb.XGBClassifier(tree_method='gpu_hist')
        
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    # Fit data to model
    history = model.fit(X_features[train], Y[train], verbose=1)
    
    # Generate generalization metrics
    #scores = model.evaluate(X_features[test], Y[test], verbose=0)
    prediction = model.predict(X_features[test])
    accuracy = metrics.accuracy_score(Y[test], prediction)
    print(f'Score for fold {fold_no}: Accuracy of {accuracy}%')
    acc_per_fold.append(accuracy * 100)
    for x in Y[test]:
        Y_actuals = np.append(Y_actuals, x)
    for z in model.predict(X_features[test]):
        Y_preds = np.append(Y_preds, np.argmax(z))
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: Accuracy of 0.921875%
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: Accuracy of 0.925%
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: Accuracy of 0.915625%
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: Accuracy of 0.92890625%
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: Accuracy of 0.9265625%
CPU times: user 2min 29s, sys: 4.04 s, total: 2min 33s
Wall time: 2min


In [14]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, 5):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Accuracy: 92.1875%
------------------------------------------------------------------------
> Fold 2 - Accuracy: 92.5%
------------------------------------------------------------------------
> Fold 3 - Accuracy: 91.5625%
------------------------------------------------------------------------
> Fold 4 - Accuracy: 92.890625%
------------------------------------------------------------------------
> Fold 5 - Accuracy: 92.65625%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 92.359375 (+- 0.4592793267718459)
------------------------------------------------------------------------
